In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import torch
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/100"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 40 # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

from transformers import AutoImageProcessor, ConvNextForImageClassification

processor = AutoImageProcessor.from_pretrained("facebook/convnext-small-224")
model = AutoModelForImageClassification.from_pretrained("facebook/convnext-small-224")

# Move model to device
model.to(device)

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Using device: cuda


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


Epoch 1/20, Train Loss: 2.5274, Train Accuracy: 39.15%, Val Loss: 1.6324, Val Accuracy: 42.72%


Epoch 2/20, Train Loss: 1.2618, Train Accuracy: 54.67%, Val Loss: 1.1700, Val Accuracy: 58.56%


Epoch 3/20, Train Loss: 0.8916, Train Accuracy: 67.65%, Val Loss: 0.9080, Val Accuracy: 68.80%


...

Epoch 16/20, Train Loss: 0.0674, Train Accuracy: 97.12%, Val Loss: 0.3209, Val Accuracy: 89.12%


Epoch 17/20, Train Loss: 0.0612, Train Accuracy: 97.53%, Val Loss: 0.3311, Val Accuracy: 88.80%


Epoch 18/20, Train Loss: 0.0641, Train Accuracy: 97.46%, Val Loss: 0.3359, Val Accuracy: 89.28%


Epoch 19/20, Train Loss: 0.0615, Train Accuracy: 97.60%, Val Loss: 0.3148, Val Accuracy: 89.12%


Epoch 20/20, Train Loss: 0.0594, Train Accuracy: 97.60%, Val Loss: 0.2903, Val Accuracy: 89.60%
Test Accuracy: 89.60%


In [8]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 89.60%
Classification Report:
              precision    recall  f1-score   support

           0      0.905     0.974     0.938        39
           1      0.882     0.789     0.833        38
           2      1.000     0.685     0.813        54
           3      0.908     0.863     0.885        80
           4      0.842     0.696     0.762        46
           5      0.910     0.947     0.928        75
           6      0.874     0.973     0.921       257
           7      0.971     0.917     0.943        36

    accuracy                          0.896       625
   macro avg      0.912     0.855     0.878       625
weighted avg      0.899     0.896     0.893       625



In [9]:
# Train the model
num_epochs = 40
for epoch in range(20, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 21/40, Train Loss: 0.0552, Train Accuracy: 97.94%, Val Loss: 0.3408, Val Accuracy: 88.80%


Epoch 22/40, Train Loss: 0.0528, Train Accuracy: 97.87%, Val Loss: 0.2975, Val Accuracy: 89.60%


Epoch 23/40, Train Loss: 0.0511, Train Accuracy: 97.80%, Val Loss: 0.3263, Val Accuracy: 88.48%


Epoch 24/40, Train Loss: 0.0506, Train Accuracy: 98.08%, Val Loss: 0.2847, Val Accuracy: 90.72%


Epoch 25/40, Train Loss: 0.0475, Train Accuracy: 97.73%, Val Loss: 0.2829, Val Accuracy: 91.04%


...

Epoch 36/40, Train Loss: 0.0407, Train Accuracy: 98.08%, Val Loss: 0.3182, Val Accuracy: 90.08%


Epoch 37/40, Train Loss: 0.0398, Train Accuracy: 97.94%, Val Loss: 0.2763, Val Accuracy: 90.72%
Epoch 00037: reducing learning rate of group 0 to 1.0000e-06.


Epoch 38/40, Train Loss: 0.0364, Train Accuracy: 98.21%, Val Loss: 0.2857, Val Accuracy: 90.40%


Epoch 39/40, Train Loss: 0.0364, Train Accuracy: 98.21%, Val Loss: 0.2905, Val Accuracy: 90.40%


Epoch 40/40, Train Loss: 0.0370, Train Accuracy: 98.15%, Val Loss: 0.2917, Val Accuracy: 90.24%
Test Accuracy: 90.24%


In [10]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 90.24%
Classification Report:
              precision    recall  f1-score   support

           0      0.905     0.974     0.938        39
           1      0.919     0.895     0.907        38
           2      1.000     0.667     0.800        54
           3      0.945     0.863     0.902        80
           4      0.882     0.652     0.750        46
           5      0.947     0.947     0.947        75
           6      0.861     0.984     0.918       257
           7      0.971     0.917     0.943        36

    accuracy                          0.902       625
   macro avg      0.929     0.862     0.888       625
weighted avg      0.908     0.902     0.899       625



In [11]:
# Train the model
num_epochs = 60
for epoch in range(40, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 41/60, Train Loss: 0.0384, Train Accuracy: 98.15%, Val Loss: 0.2968, Val Accuracy: 90.08%
Epoch 00041: reducing learning rate of group 0 to 1.0000e-07.


Epoch 42/60, Train Loss: 0.0380, Train Accuracy: 97.94%, Val Loss: 0.2968, Val Accuracy: 90.24%


Epoch 43/60, Train Loss: 0.0375, Train Accuracy: 98.21%, Val Loss: 0.2971, Val Accuracy: 90.24%


Epoch 44/60, Train Loss: 0.0399, Train Accuracy: 98.28%, Val Loss: 0.2972, Val Accuracy: 90.24%


Epoch 45/60, Train Loss: 0.0367, Train Accuracy: 97.87%, Val Loss: 0.2969, Val Accuracy: 90.24%
Epoch 00045: reducing learning rate of group 0 to 1.0000e-08.


...

Epoch 56/60, Train Loss: 0.0369, Train Accuracy: 98.35%, Val Loss: 0.2967, Val Accuracy: 90.24%


Epoch 57/60, Train Loss: 0.0347, Train Accuracy: 98.56%, Val Loss: 0.2967, Val Accuracy: 90.24%


Epoch 58/60, Train Loss: 0.0349, Train Accuracy: 98.49%, Val Loss: 0.2967, Val Accuracy: 90.24%


Epoch 59/60, Train Loss: 0.0378, Train Accuracy: 98.28%, Val Loss: 0.2966, Val Accuracy: 90.24%


Epoch 60/60, Train Loss: 0.0362, Train Accuracy: 98.15%, Val Loss: 0.2966, Val Accuracy: 90.24%
Test Accuracy: 90.24%


In [12]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 90.24%
Classification Report:
              precision    recall  f1-score   support

           0      0.905     0.974     0.938        39
           1      0.919     0.895     0.907        38
           2      1.000     0.648     0.787        54
           3      0.945     0.863     0.902        80
           4      0.886     0.674     0.765        46
           5      0.947     0.947     0.947        75
           6      0.861     0.984     0.918       257
           7      0.971     0.917     0.943        36

    accuracy                          0.902       625
   macro avg      0.929     0.863     0.888       625
weighted avg      0.908     0.902     0.899       625

